In [1]:
import numpy as np
import scipy as sc
import mpmath as mm
import decimal as dc
import sympy as sp
import matplotlib as mp
import matplotlib.pyplot as plt
import memory_profiler
import time
import os
from scipy import linalg
from mpmath import mpf, mp
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
from scipy.sparse import bsr_matrix

In [2]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

#### Finite difference for PDE

In [3]:
class Propagator(object):
    def __init__(self, model, dx, F, npad = 1):
        self.nx           = len(model)
        self.dx           = np.float64(dx)
        self.nx_padded    = self.nx + 2 * npad
        self.nx_padded_   = self.nx + 2 * (npad + 1)
        self.r            = model
        self.model_padded = np.pad(self.r, (npad, npad), 'edge')
        self.F_           = F
        self.F            = np.pad(self.F_, (npad, npad), 'edge')
        self.field        = [np.zeros(self.nx_padded),
                             np.zeros(self.nx_padded)]
        self.Dx           = self.field[0]
        self.Dxx          = self.field[1]

class VPy1(Propagator):
    """A simple Python implementation."""
    def __init__(self, model, dx, F):
        super(VPy1, self).__init__(model, dx, F, npad = 1)
        """Propagate wavefield."""
        f    = self.F
        f_xx = self.Dxx
        f_x  = self.Dx

        for x in range(1, self.nx_padded-1):
#             f_xx[x] = (1*f[x-1]-2*f[x+0]+1*f[x+1])/(self.dx**2)
#             f_x[x]  = (-1*f[x-1]+1*f[x+1])/(2*self.dx)
            
            if x == 1:
                f_xx[x] = (2*f[x+0]-5*f[x+1]+4*f[x+2]-1*f[x+3])/(self.dx**2)
                f_x[x]  = (-11*f[x+0]+18*f[x+1]-9*f[x+2]+2*f[x+3])/(6*self.dx)
            elif x > 1 and x != self.nx_padded-2:
                f_xx[x] = (1*f[x-1]-2*f[x+0]+1*f[x+1])/(self.dx**2)
                f_x[x]  = (-1*f[x-1]+1*f[x+1])/(2*self.dx)
            else:
                f_xx[x] = (-1*f[x-3]+4*f[x-2]-5*f[x-1]+2*f[x+0])/(self.dx**2)
                f_x[x]  = (-2*f[x-3]+9*f[x-2]-18*f[x-1]+11*f[x+0])/(6*self.dx)

        self.f_x  = f_x
        self.f_xx = f_xx
        self.u_rr = f_xx + ((2.0 / self.model_padded) * f_x) - ((2.0 / self.model_padded**2) * f)
        self.p_rr = f_xx + (2.0 / self.model_padded) * f_x

class D(VPy1):
    def __init__(self, model, dx, F):
        super(D, self).__init__(model, dx, F)
        self.f_x  = self.f_x[1:self.nx_padded-1]
        self.f_xx = self.f_xx[1:self.nx_padded-1]
        self.u_rr = self.u_rr[1:self.nx_padded-1]
        self.p_rr = self.p_rr[1:self.nx_padded-1]
        
    def DF(self):
        yield self.f_x
    
    def DF_2(self):
        yield self.f_xx
    
    def DU_2(self):
        yield self.u_rr

    def DP_2(self):
        yield self.p_rr

#### Loss function and gradient

In [4]:
def gradient(Func, Data, K, t):
    S = ['a', 'c', 'v', 'e']
    I = [  0,   3,   2,   2]
    J = [  1,   1,   1,   3]
    T = ['ac', 'ec', 'vc', 've']
    
    for x in range(4):
        if t == T[x]:
            j = I[x]
            i = J[x]
        elif t[::-1] == T[x]:
            j = I[x]
            i = J[x]
    
    DP    = next(D(r, dr, Data[:, j]).DP_2())
    DA_p  = K * DP - np.matmul(Func, P)[j]
    grad_ = DA_p * (Dat[:,i] - Dat[:,j])
    grad  = np.sum(grad_) / Num
    
    yield grad
    del S, I, DP, DA_p, cost, grad_

#### Optimizer

In [5]:
def Optimizer(gamma, grad, n, d, opt, t, Error = None):
    S = ['a', 'c', 'v', 'e']
    I = [  0,   1,   2,   3]
    C = [  1,   3,   4,   5]
    J = [  0,   2,   4,   6]
    
    for x in range(4):
        if t[2] == S[x]:
            j = I[x]

        if t[3] == S[x]:
            i = I[x]
    
    if opt == 'GD':
        alpha = 1.0e-9
        gamma = gamma - (alpha * grad)
        n     = n
        yield gamma, n
        
    if opt == 'AdaGrad':
#         eta = 1.0e-10
        if d == 0:
            eta = 1.0e-10
            
        else:
            d_ = d // 10
            eta = 10**(- 0.5 * d_ - 10)
            if (gamma / eta) < 10:
                eta = eta / 10
        
#         eta = gamma / 1.0e2
#         for q in range(4):
#             eta = gamma / 1.0e2
#             if (j + i) == C[q]:
#                 if j < i:
#                     eta = gamma[J[q]] / 1.0e2
                
#                 else:
#                     eta = gamma[J[q] + 1] / 1.0e2
        
        e     = 1.0e-20
        n     = n + np.square(grad)
        sqrt  = 1 / np.sqrt(n + e)
        grad_ = sqrt * grad
        gamma = gamma - (eta * grad_)
#         if Error == None:
#             pass

#         else:
#             if np.ceil(Error/epsilon) > 1:
#                 if gamma > eta * grad_:
#                     gamma = gamma - (eta * grad_)
                    
#                     if np.ceil(gamma / (eta * grad_)) > 100 and np.ceil(gamma / (eta * grad_)) < 10000:
#                         eta = eta * 1.0e2
#                         gamma = gamma - (eta * grad_)
                        
#                     elif np.ceil(gamma / (eta * grad_)) > 10000:
#                         eta = eta * 1.0e4
#                         gamma = gamma - (eta * grad_)
                        
#                 else:
#                     exp   = np.ceil(np.log10((eta * grad_) / gamma))
#                     eta   = eta * 10**(- exp)
#                     if np.ceil(gamma / (eta * grad_)) < 10:
#                         eta = eta * 1.0e-2
#                         gamma = gamma - (eta * grad_)
                        
#                     else:
#                         gamma = gamma - (eta * grad_)
                        
#             elif np.log10(Error/epsilon) < 0:
#                 eta   = eta * 1.0e-20
#                 gamma = gamma - (eta * grad_)
        
        # if gamma > eta * grad_:
        #     gamma = gamma - (eta * grad_)
        #     if np.ceil(gamma / (eta * grad_)) > 100 and np.ceil(gamma / (eta * grad_)) < 10000:
        #         eta = eta * 1.0e2
        #         gamma = gamma - (eta * grad_)

        #     elif np.ceil(gamma / (eta * grad_)) > 10000:
        #     	eta = eta * 1.0e4
        #     	gamma = gamma - (eta * grad_)
            
        # else:
        #     exp   = np.ceil(np.log10((eta * grad_) / gamma))
        #     eta   = eta * 10**(- exp)
        #     if np.ceil(gamma / (eta * grad_)) < 10:
        #         eta = eta * 1.0e-2
        #         gamma = gamma - (eta * grad_)
            
        #     else:
        #         gamma = gamma - (eta * grad_)
            
        yield gamma, n
        
    del S, I, sqrt, grad_

#### Model

In [6]:
def Model(Data, gamma, grad, n, epoch, d = 0, Error = [None, None, None, None, None, None, None, None]):
    if np.equal(gamma, None).all() and np.equal(grad, None).all():
#         R_ = np.random.rand(1)
#         a, b, c, d = R_[0] / 1.0e10, R_[0] / 1.0e10, R_[0] / 1.0e14, R_[0] / 1.0e10
        a, b, c, d = np.random.uniform(0.1, 0.2, size = (4)) / (1.0e9)
        gamma[0], gamma[1], gamma[2], gamma[3], gamma[4], gamma[5], gamma[6], gamma[7] = a, a, b, b, c, c, d, d
#         a1, a2, a3, a4, a5, a6, a7, a8 = np.random.uniform(0.1, 0.2, size = (8)) / (1.0e9)
#         gamma[0], gamma[1], gamma[2], gamma[3], gamma[4], gamma[5], gamma[6], gamma[7] = a1, a2, a3, a4, a5, a6, a7, a8
#         gamma = gamma_
    elif np.not_equal(gamma, None).all() and np.not_equal(grad, None).all():
        Bool               = np.greater(gamma, 0)
        Index              = np.array([[0, 1], [2, 3], [4, 5], [6, 7]])
        Str                = ['r_ac', 'r_ca', 'r_cv', 'r_vc', 'r_ce', 'r_ec', 'r_ve', 'r_ev']
        
        for i in range(8):
            j = np.where(Index == i)[0][0]# Error 4 to 8
            if Bool[i] and np.greater(Error[j], epsilon):
                gamma[i], n[i] = next(Optimizer(gamma[i], grad[i], n[i], d, 'AdaGrad', Str[i]))
            else:
                gamma[i], n[i] = gamma[i], n[i]
                
#         if np.greater(Error[0], epsilon):
#             gamma[0], n[0] = next(Optimizer(gamma[0], grad[0], n[0], d, 'AdaGrad', 'r_ac'))
#             gamma[1], n[1] = next(Optimizer(gamma[1], grad[1], n[1], d, 'AdaGrad', 'r_ca'))
            
#         else:
#             gamma[0], n[0] = gamma[0], n[0]
#             gamma[1], n[1] = gamma[1], n[1]
            
#         if np.greater(Error[1], epsilon):
#             gamma[4], n[4] = next(Optimizer(gamma[4], grad[4], n[4], d, 'AdaGrad', 'r_ce'))
#             gamma[5], n[5] = next(Optimizer(gamma[5], grad[5], n[5], d, 'AdaGrad', 'r_ec'))
            
#         else:
#             gamma[4], n[4] = gamma[4], n[4]
#             gamma[5], n[5] = gamma[5], n[5]
        
#         if np.greater(Error[2], epsilon):
#             gamma[2], n[2] = next(Optimizer(gamma[2], grad[2], n[2], d, 'AdaGrad', 'r_cv'))
#             gamma[3], n[3] = next(Optimizer(gamma[3], grad[3], n[3], d, 'AdaGrad', 'r_vc'))
            
#         else:
#             gamma[2], n[2] = gamma[2], n[2]
#             gamma[3], n[3] = gamma[3], n[3]
        
#         if np.greater(Error[3], epsilon):
#             gamma[6], n[6] = next(Optimizer(gamma[6], grad[6], n[6], d, 'AdaGrad', 'r_ve'))
#             gamma[7], n[7] = next(Optimizer(gamma[7], grad[7], n[7], d, 'AdaGrad', 'r_ev'))
            
#         else:
#             gamma[6], n[6] = gamma[6], n[6]
#             gamma[7], n[7] = gamma[7], n[7]
    else:
        pass
    
    A              = [[   gamma[0],                     - gamma[0],                   0,                   0],
                      [ - gamma[1], gamma[1] + gamma[2] + gamma[4],          - gamma[2],          - gamma[4]],
                      [          0,                     - gamma[3], gamma[3] + gamma[6],          - gamma[6]],
                      [          0,                     - gamma[5],          - gamma[7], gamma[5] + gamma[7]]]
    
    grad[0]        = next(gradient(A, Data, ka, 'ac')) # for loop
    grad[1]        = next(gradient(A, Data, kc, 'ca'))
    grad[2]        = next(gradient(A, Data, kc, 'cv'))
    grad[3]        = next(gradient(A, Data, kv, 'vc'))
    grad[4]        = next(gradient(A, Data, kc, 'ce'))
    grad[5]        = next(gradient(A, Data, ke, 'ec'))
    grad[6]        = next(gradient(A, Data, kv, 've'))
    grad[7]        = next(gradient(A, Data, ke, 'ev'))
    
    yield gamma, grad, n
    del A

#### Enumerate

In [7]:
def Enumerate(Error):
    l = 0
    for l, n in enumerate(Error):
        if n == np.min(Error):
            break
    
    return l

#### File saver

In [8]:
def Save(gamma, grad, e):
    Com      = ['ac', 'ca', 'cv', 'vc', 'ce', 'ec', 've', 'ev']
    createFolder('./data  general/iteration {}'.format(e))
    for s in range(8):
        str1 = 'data  general/iteration {}/r_{}_t_iteration_{}'.format(e, Com[s], e)
        str2 = 'data  general/iteration {}/grad_{}_t_iteration_{}'.format(e, Com[s], e)
        np.savetxt(str1, gamma[s])
        np.savetxt(str2, grad[s])

#### Information

In [9]:
def info(epochs, epsilons, times, memories):
    Times = []
    for i in range(np.shape(times)[0]):
        if i == 0:
            Times.append(times[i] - tStart)
        else:
            Times.append(times[i] - times[i - 1])
    
    Mem   = []
    for j in range(np.shape(memories)[0]):
        if j == 0:
            Mem.append(memories[i] - mem1[0])
        else:
            Mem.append(memories[i] - memories[i - 1])
    
    loca  = '{0:15}'
    f     = open("data  general/Readme.txt","w+")
    f.write('{0[0]:<15}{1[0]:<15}\n'.format(['Version'], ['5 iterations 10000 nodes, full code, ver 2']))
    f.write('{0:15}{1}\n'.format('Iterations', e+1))
    f.write(loca.format('Epsilon'))
    s     = ''
    for i in range(0, e + 1):
        ss    = '0:20'
        s     = '{' + ss + '},'
        f.write(s.format(epsilons[i]))
    
    f.write('\n')
    f.write(loca.format('Epochs'))
    s     = ''
    for i in range(0, e + 1):
        ss    = '0:20'
        s     = '{' + ss + '},'
        f.write(s.format(epochs[i]))
        
    f.write('\n')
    f.write(loca.format('Times'))
    s     = ''
    for i in range(0, e + 1):
        ss    = '0:20'
        s     = '{' + ss + '},'
        f.write(s.format(Times[i]))
        
    f.write('\n')
    f.write(loca.format('Memories'))
    s     = ''
    for i in range(0, e + 1):
        ss    = '0:20'
        s     = '{' + ss + '},'
        f.write(s.format(Mem[i]))

#### Train

In [10]:
def train(param, Grad, M, epoch):
    tStart              = time.time()
    time.sleep(2)
#     print(tStart)
    Break               = []
    flag                = [     0,      0,      0,      0,      0,      0,      0,      0]
    
    if e == 0:
        gamma, grad, n      = next(Model(Dat, param, Grad, M, epoch))
        
        r_ac_t              = [gamma[0]]
        r_ca_t              = [gamma[1]]
        r_cv_t              = [gamma[2]]
        r_vc_t              = [gamma[3]]
        r_ce_t              = [gamma[4]]
        r_ec_t              = [gamma[5]]
        r_ve_t              = [gamma[6]]
        r_ev_t              = [gamma[7]]
        
        grad_ac_t           = [grad[0]]
        grad_ca_t           = [grad[1]]
        grad_cv_t           = [grad[2]]
        grad_vc_t           = [grad[3]]
        grad_ce_t           = [grad[4]]
        grad_ec_t           = [grad[5]]
        grad_ve_t           = [grad[6]]
        grad_ev_t           = [grad[7]]
        
#         Errorac_t           = []
#         Errorca_t           = []
#         Errorcv_t           = []
#         Errorvc_t           = []
#         Errorce_t           = []
#         Errorec_t           = []
#         Errorve_t           = []
#         Errorev_t           = []
        Errora_t             = []
        Errorc_t             = []
        Errorv_t             = []
        Errore_t             = []

    DPa                 = next(D(r, dr, Dat[:, 0]).DP_2())
    DPc                 = next(D(r, dr, Dat[:, 1]).DP_2())
    DPv                 = next(D(r, dr, Dat[:, 2]).DP_2())
    DPe                 = next(D(r, dr, Dat[:, 3]).DP_2())

    KDPa                = ka * DPa
    KDPc                = kc * DPc
    KDPv                = kv * DPv
    KDPe                = ke * DPe
    p_ac                = Dat[:, 0] - Dat[:, 1]
    p_ca                = Dat[:, 1] - Dat[:, 0]
    p_cv                = Dat[:, 1] - Dat[:, 2]
    p_vc                = Dat[:, 2] - Dat[:, 1]
    p_ce                = Dat[:, 1] - Dat[:, 3]
    p_ec                = Dat[:, 3] - Dat[:, 1]
    p_ve                = Dat[:, 2] - Dat[:, 3]
    p_ev                = Dat[:, 3] - Dat[:, 2]

    epochs              = []
    epsilons            = []
    times               = []
    memories            = []

    createFolder('./data  general/gamma/')

# for e in range(5):
#     epsilon = 10**(-2 * e - 6)
#     eta     = 10**(-2 * e - 13)
    epsilons.append(epsilon)
    print("Iteration", e)
    np.savetxt('data  general/gamma/iteration_{}.dat'.format(e), [param])
    f = open('data  general/gamma/iteration_{}.dat'.format(e), 'a+')

    if e > 0:
        epochs.append(epoch)
        epoch          = 0
        
        gamma          = param

        r_ac_t         = [gamma[0]]
        r_ca_t         = [gamma[1]]
        r_cv_t         = [gamma[2]]
        r_vc_t         = [gamma[3]]
        r_ce_t         = [gamma[4]]
        r_ec_t         = [gamma[5]]
        r_ve_t         = [gamma[6]]
        r_ev_t         = [gamma[7]]

#         Errorac_t      = []
#         Errorca_t      = []
#         Errorcv_t      = []
#         Errorvc_t      = []
#         Errorce_t      = []
#         Errorec_t      = []
#         Errorve_t      = []
#         Errorev_t      = []
        Errora_t       = []
        Errorc_t       = []
        Errorv_t       = []
        Errore_t       = []

        n              = [ 0, 0, 0, 0, 0, 0, 0, 0]
        
        gamma, grad, n = next(Model(Dat, param, Grad, M, epoch, d = e))
        
        grad_ac_t      = [grad[0]]
        grad_ca_t      = [grad[1]]
        grad_cv_t      = [grad[2]]
        grad_vc_t      = [grad[3]]
        grad_ce_t      = [grad[4]]
        grad_ec_t      = [grad[5]]
        grad_ve_t      = [grad[6]]
        grad_ev_t      = [grad[7]]

    while epoch >= 0:
        try:
            # change to the difference between analytical data and predicted data
            Errora = np.mean(np.abs(KDPa - gamma[0] * p_ac)) 
            Errorc = np.mean(np.abs(KDPc - gamma[1] * p_ca - gamma[2] * p_cv - gamma[4] * p_ce))
            Errorv = np.mean(np.abs(KDPv - gamma[3] * p_vc - gamma[6] * p_ve))
            Errore = np.mean(np.abs(KDPe - gamma[5] * p_ec - gamma[7] * p_ev))
#             Errorac                   = np.mean(np.abs(KDPa - gamma[0] * p_ac)) 
#             Errorca, Errorcv, Errorce = np.mean(np.abs(KDPc - gamma[1] * p_ca - gamma[2] * p_cv - gamma[4] * p_ce)), np.mean(np.abs(KDPc - gamma[1] * p_ca - gamma[2] * p_cv - gamma[4] * p_ce)), np.mean(np.abs(KDPc - gamma[1] * p_ca - gamma[2] * p_cv - gamma[4] * p_ce))
#             Errorvc, Errorve          = np.mean(np.abs(KDPv - gamma[3] * p_vc - gamma[6] * p_ve)), np.mean(np.abs(KDPv - gamma[3] * p_vc - gamma[6] * p_ve))
#             Errorec, Errorev          = np.mean(np.abs(KDPe - gamma[5] * p_ec - gamma[7] * p_ev)), np.mean(np.abs(KDPe - gamma[5] * p_ec - gamma[7] * p_ev))
            Stda                      = np.std(KDPa - gamma[0] * p_ac)
            Stdc                      = np.std(KDPc - gamma[1] * p_ca - gamma[2] * p_cv - gamma[4] * p_ce)
            Stdv                      = np.std(KDPv - gamma[3] * p_vc - gamma[6] * p_ve)
            Stde                      = np.std(KDPe - gamma[5] * p_ec - gamma[7] * p_ev)
            Error_                    = [Errora, Errorc, Errorv, Errore]
#             Error_                    = [Errorac, Errorca, Errorcv, Errorvc, Errorce, Errorec, Errorve, Errorev]
            
            r_list                    = [r_ac_t, r_ca_t, r_cv_t, r_vc_t, r_ce_t, r_ec_t, r_ve_t, r_ev_t]

#             if epoch % 20 == 0:
#                 print(epoch)
            if np.greater(Errora, epsilon) or np.greater(Errorc, epsilon) or np.greater(Errorv, epsilon) or np.greater(Errore, epsilon):
#             if np.greater(Errorac, epsilon) or np.greater(Errorca, epsilon) or np.greater(Errorcv, epsilon) or np.greater(Errorvc, epsilon) or np.greater(Errorce, epsilon) or np.greater(Errorec, epsilon) or np.greater(Errorve, epsilon) or np.greater(Errorev, epsilon):
#             if epoch < epochs:
                gamma, grad, n = next(Model(Dat, gamma, grad, M, epoch, d = e, Error = Error_))
                
                if np.less(gamma, 0).any() or np.equal(flag, 1).any():
                    for i in range(8):
                        if gamma[i] < 0 or flag[i] == 1:
                            gamma[i] = r_list[i][-1]
                            flag[i]  = 1

                grad_ac_t.append(grad[0])
                grad_ca_t.append(grad[1])
                grad_cv_t.append(grad[2])
                grad_vc_t.append(grad[3])
                grad_ce_t.append(grad[4])
                grad_ec_t.append(grad[5])
                grad_ve_t.append(grad[6])
                grad_ev_t.append(grad[7])

                r_ac_t.append(gamma[0])
                r_ca_t.append(gamma[1])
                r_cv_t.append(gamma[2])
                r_vc_t.append(gamma[3])
                r_ce_t.append(gamma[4])
                r_ec_t.append(gamma[5])
                r_ve_t.append(gamma[6])
                r_ev_t.append(gamma[7])

                Errora_t.append(Errora)
                Errorc_t.append(Errorc)
                Errorv_t.append(Errorv)
                Errore_t.append(Errore)
#                 Errorac_t.append(Errorac)
#                 Errorca_t.append(Errorca)
#                 Errorcv_t.append(Errorcv)
#                 Errorvc_t.append(Errorvc)
#                 Errorce_t.append(Errorce)
#                 Errorec_t.append(Errorec)
#                 Errorve_t.append(Errorve)
#                 Errorev_t.append(Errorev)
                
                r_list    = [r_ac_t, r_ca_t, r_cv_t, r_vc_t, r_ce_t, r_ec_t, r_ve_t, r_ev_t]
                
                for b in range(8):
                    f.writelines("%.18e" % gamma[b])
                    f.writelines(" ")
                f.writelines("\n")

#                 if np.less(gamma, 0).any():
#                     Bool = np.less(gamma, 0)
#                     Nega = sum(Bool)
                    
#                     memiter   = memory_profiler.memory_usage()
#                     titer     = time.time()
#                     memories.append(memiter[0])
#                     times.append(titer)
#                     r_list    = [r_ac_t, r_ca_t, r_cv_t, r_vc_t, r_ce_t, r_ec_t, r_ve_t, r_ev_t]
#                     grad_list = [grad_ac_t, grad_ca_t, grad_cv_t, grad_vc_t, grad_ce_t, grad_ec_t, grad_ve_t, grad_ev_t]
#                     Save(r_list, grad_list, e)
#                     print('epsilon = ', epsilon, epoch)#1.0e(-2 * e - 6)
#                     print(f"negative values appear {Nega} times.")
#                     break
                
                for i in range(8):
                    Break.append(len(set(r_list[i][-3:])))
                
                if np.equal(Break, 1).all():
                    print(gamma) # Add if condition to modify gamma
                    print('epsilon = ', epsilon, epoch)
                    break
                
                else:
                    Break = []
            
            else:
                memiter   = memory_profiler.memory_usage()
                titer     = time.time()
                memories.append(memiter[0])
                times.append(titer)
                r_list    = [r_ac_t, r_ca_t, r_cv_t, r_vc_t, r_ce_t, r_ec_t, r_ve_t, r_ev_t]
                grad_list = [grad_ac_t, grad_ca_t, grad_cv_t, grad_vc_t, grad_ce_t, grad_ec_t, grad_ve_t, grad_ev_t]
                Save(r_list, grad_list, e)
                del r_list, grad_list
                print(gamma) # Add if condition to modify gamma
                print('epsilon = ', epsilon, epoch)#1.0e(-2 * e - 6)
                break

            epoch += 1
        except KeyboardInterrupt:
            memInterrupt = memory_profiler.memory_usage()
            tInterrupt   = time.time()
            mem_diff     = memInterrupt[0] - mem1[0]
            time_diff    = tInterrupt - tStart
#             info(epochs, epsilons, times, memories)
            print(f"It took {time_diff} Secs and {mem_diff} Mb to execute this method till interrupted")
            raise

#     if np.less(gamma, 0).any():
#         print(gamma)
#         print('epsilon = 1.0e', (-2 * e - 6), epoch)
#         print('negative values appear')
#         break
    
    print(len(Errora_t))
    print(Enumerate(Errora_t), Enumerate(Errorc_t), Enumerate(Errorv_t), Enumerate(Errore_t))
#     print(Enumerate(Errorac_t), Enumerate(Errorca_t), Enumerate(Errorcv_t), Enumerate(Errorvc_t), Enumerate(Errorce_t), Enumerate(Errorec_t), Enumerate(Errorve_t), Enumerate(Errorev_t))
    gamma = [r_ac_t[Enumerate(Errora_t)], r_ca_t[Enumerate(Errorc_t)], r_cv_t[Enumerate(Errorc_t)], r_vc_t[Enumerate(Errorv_t)], r_ce_t[Enumerate(Errorc_t)], r_ec_t[Enumerate(Errore_t)], r_ve_t[Enumerate(Errorv_t)], r_ev_t[Enumerate(Errore_t)]]
    for i in np.linspace(0, 6, 4, dtype = int):
        if i < 6:
            if gamma[i] > gamma[i + 1]:
                gamma[i] = gamma[i + 1]
            else:
                gamma[i + 1] = gamma[i]
        else:
            if gamma[i] > gamma[i + 1]:
                gamma[i + 1] = gamma[i]
            else:
                gamma[i] = gamma[i + 1]
    grad  = [None, None, None, None, None, None, None, None]
#     f.close()

    epochs.append(epoch)
    mem2      = memory_profiler.memory_usage()
    tEnd      = time.time()
    mem_diff  = mem2[0] - mem1[0]
    time_diff = tEnd - tStart
#     info(epochs, epsilons, times, memories)
    print(f"It took {time_diff} Secs and {mem_diff} Mb to execute this method")
    
    return gamma, grad

#### Test

In [22]:
def test(gamma):
    PAA     = np.loadtxt('/Users/jasonwang/Desktop/Lab/datasets/pa_test.dat')
    PCC     = np.loadtxt('/Users/jasonwang/Desktop/Lab/datasets/pc_test.dat')
    PVV     = np.loadtxt('/Users/jasonwang/Desktop/Lab/datasets/pv_test.dat')
    PEE     = np.loadtxt('/Users/jasonwang/Desktop/Lab/datasets/pe_test.dat')

    r       = np.linspace(0.03, 0.1, 10000)
    r_ana   = np.linspace(0.03, 0.1, len(PAA))
    PA_ana  = np.reshape(np.interp(r, r_ana, np.reshape(PAA, (len(PAA), ))), (10000, 1))
    PC_ana  = np.reshape(np.interp(r, r_ana, np.reshape(PCC, (len(PCC), ))), (10000, 1))
    PV_ana  = np.reshape(np.interp(r, r_ana, np.reshape(PVV, (len(PVV), ))), (10000, 1))
    PE_ana  = np.reshape(np.interp(r, r_ana, np.reshape(PEE, (len(PEE), ))), (10000, 1))

    PI      = np.pi
    r_ac    = gamma[0]
    r_ca    = gamma[1]
    r_cv    = gamma[2]
    r_vc    = gamma[3]
    r_ce    = gamma[4]
    r_ec    = gamma[5]
    r_ve    = gamma[6]
    r_ev    = gamma[7]

#     r_ac    = 1.5e-19
#     r_ca    = 1.5e-19
#     r_cv    = 1.5e-19
#     r_vc    = 1.5e-19
#     r_ce    = 1.0e-22
#     r_ec    = 1.0e-22
#     r_ev    = 1.0e-13
#     r_ve    = 1.0e-13
    L       = 7.0e-2
    d       = 4.0e-3
    Q_p     = 5.8e-9
    R_cp    = 6.0e-4
    Ka      = 1.0e-10
    Kc      = 1.0e-10
    Kv      = 1.0e-10
    Ke      = 1.4e-14
    Ma      = 2.67e-3
    Mc      = 2.67e-3
    Mv      = 2.67e-3
    Me      = 8.9e-4
    ka      = Ka / Ma
    kc      = Kc / Mc
    kv      = Kv / Mv
    ke      = Ke / Me
    H       = (PI * d**4) / (128.0 * Me * L)
    E       = 584.0
    nu      = 0.35
    G       = E / (2.0 * (1.0 + nu))
    alpha_a = 1.0
    alpha_c = 1.0
    alpha_v = 1.0
    alpha_e = 1.0

    c1      = sp.Symbol('c1' )
    c2      = sp.Symbol('c2' )
    c3      = sp.Symbol('c3' )
    c4      = sp.Symbol('c4' )
    c5      = sp.Symbol('c5' )
    c6      = sp.Symbol('c6' )
    c7      = sp.Symbol('c7' )
    c8      = sp.Symbol('c8' )
    c9      = sp.Symbol('c9' )
    c10     = sp.Symbol('c10')
    r       = sp.Symbol('r'  )

    A       = np.array([[ r_ac / ka,               - r_ac / ka,                   0.0,                   0.0],
                        [-r_ca / kc, (r_ca + r_cv + r_ce) / kc,           - r_cv / kc,           - r_ce / kc],
                        [       0.0,               - r_vc / kv,    (r_vc + r_ve) / kv,           - r_ve / kv],
                        [       0.0,               - r_ec / ke,           - r_ev / ke,    (r_ec + r_ev) / ke]])
    val,vec = linalg.eig(A)
    M       = np.argsort(val)
    K       = np.array([val[M[0]], val[M[3]], val[M[2]], val[M[1]]])
    F       = sp.Matrix([0, val[M[3]], val[M[2]], val[M[1]]])
    V       = sp.Matrix(np.array([-vec[:, M[0]], -vec[:, M[3]], -vec[:, M[2]], -vec[:, M[1]]]).T)
    Vector  = sp.Matrix(V).evalf(digs)

    Value   = sp.Matrix(K)
    Value   = Value.evalf(digs)
    l       = np.array(Value)
    L       = np.diag(l)
    Q       = np.array(Vector)
    B       = sp.Matrix(Q).evalf(digs)
#     l[0]    = 0
    q       = sp.Matrix([[c2 + c1/r],
                         [(c3 * sp.sinh(r * sp.sqrt(l[1])) + c4 * sp.cosh(r * sp.sqrt(l[1]))) / r],
                         [(c5 * sp.sinh(r * sp.sqrt(l[2])) + c6 * sp.cosh(r * sp.sqrt(l[2]))) / r],
                         [(c7 * sp.sinh(r * sp.sqrt(l[3])) + c8 * sp.cosh(r * sp.sqrt(l[3]))) / r]]).evalf(digs)
    P       = (B * q).evalf(digs)

#     F       = sp.Matrix([[c1 / r],
#                          [(sp.sqrt(l[1]) * c3 * (sp.sqrt(l[1]) * r * sp.sinh(sp.sqrt(l[1]) * r) - sp.cosh(sp.sqrt(l[1]) * r)) + sp.sqrt(l[1]) * c4 * (sp.sqrt(l[1]) * r * sp.cosh(sp.sqrt(l[1]) * r) - sp.sinh(sp.sqrt(l[1]) * r))) / ((sp.sqrt(l[1]))**3 * r)],
#                          [(sp.sqrt(l[2]) * c5 * (sp.sqrt(l[2]) * r * sp.sinh(sp.sqrt(l[2]) * r) - sp.cosh(sp.sqrt(l[2]) * r)) + sp.sqrt(l[2]) * c6 * (sp.sqrt(l[2]) * r * sp.cosh(sp.sqrt(l[2]) * r) - sp.sinh(sp.sqrt(l[2]) * r))) / ((sp.sqrt(l[2]))**3 * r)],
#                          [(sp.sqrt(l[3]) * c7 * (sp.sqrt(l[3]) * r * sp.sinh(sp.sqrt(l[3]) * r) - sp.cosh(sp.sqrt(l[3]) * r)) + sp.sqrt(l[3]) * c8 * (sp.sqrt(l[3]) * r * sp.cosh(sp.sqrt(l[3]) * r) - sp.sinh(sp.sqrt(l[3]) * r))) / ((sp.sqrt(l[3]))**3 * r)]]).evalf(digs)
    F       = sp.diff(q, r).evalf(digs)
    Alpha   = np.array([[alpha_a], [alpha_c], [alpha_v], [alpha_e]])
    Alpha1  = Alpha.reshape(1, 4)
    Alpha1  = sp.Matrix(Alpha1)
    DD      = ((sp.N(1.0, digs) - sp.N(2.0, digs) * nu) / (sp.N(2.0, digs) * G * (sp.N(1.0, digs) - nu))) * Alpha1# * B
    U       = DD * (B * F)

    p       = sp.Matrix([[sp.N(0.0, digs)], [sp.N(0.0, digs)], [sp.N(0.0, digs)], [sp.N(0.0, digs)], [sp.N(0.0, digs)]]).evalf(digs)
    P4      = sp.Matrix([(c9 * r + c10 / r**2) + U[0]]).evalf(digs)
    p[0]    = (sp.diff(P[0], r).subs(r, sp.N(0.03, digs))).evalf(digs)
    p[1]    = (sp.diff(P[1], r).subs(r, sp.N(0.03, digs)) - (Q_p / R_cp)).evalf(digs)
    p[2]    = (sp.diff(P[2], r).subs(r, sp.N(0.03, digs))).evalf(digs)
    p[3]    = (P[3].subs(r, sp.N(0.03, digs)) - (Q_p + H * (P[3].subs(r, sp.N(0.1, digs))) + (sp.N(4.0, digs) * sp.N(PI, digs) * ke * (sp.N(0.03, digs))**2) * (sp.diff(P[3], r).subs(r, sp.N(0.03, digs))).evalf(digs)) / H).evalf(digs)#- (P[3].subs(r, sp.N(0.03, digs)))
    p[4]    = ((sp.N(1.0, digs) - nu) * sp.diff(P4, r).subs(r, sp.N(0.03, digs)) + (sp.N(2.0, digs) * nu / (sp.N(0.03, digs))) * P4.subs(r, sp.N(0.03, digs))).evalf(digs)
    p_ven   = sp.Matrix(p)

    p1      = sp.Matrix([[sp.N(0.0, digs)], [sp.N(0.0, digs)], [sp.N(0.0, digs)], [sp.N(0.0, digs)], [sp.N(0.0, digs)]]).evalf(digs)
    p1[0]   = (P[0].subs(r, sp.N(0.1, digs)) - sp.N(13333.333333, digs)).evalf(digs)
    p1[1]   = (P[1].subs(r, sp.N(0.1, digs)) - sp.N(3333.333333, digs)).evalf(digs)
    p1[2]   = (P[2].subs(r, sp.N(0.1, digs)) - sp.N(650.0, digs)).evalf(digs)
    p1[3]   = (P[3].subs(r, sp.N(0.1, digs)) - sp.N(1088.77, digs)).evalf(digs)
    p1[4]   = (P4.subs(r, sp.N(0.1, digs))).evalf(digs)
    p_sk    = sp.Matrix(p1)

    ans1    = sp.solve(p_sk, [c2, c4, c6, c8, c10], rational = True)
    p_v     = p_ven.subs([(c2, ans1[c2]), (c4, ans1[c4]), (c6, ans1[c6]), (c8, ans1[c8]), (c10, ans1[c10])])
    ans3    = sp.solve(p_v, [c1, c3, c5, c7, c9], rational = True)
    Pj      = P.subs([(c2, ans1[c2]), (c4, ans1[c4]), (c6, ans1[c6]), (c8, ans1[c8]), (c10, ans1[c10]), (c3, ans3[c3]), (c5, ans3[c5]), (c7, ans3[c7]), (c9, ans3[c9]), (c1, ans3[c1])])
    u       = P4.subs([(c2, ans1[c2]), (c4, ans1[c4]), (c6, ans1[c6]), (c8, ans1[c8]), (c10, ans1[c10]), (c3, ans3[c3]), (c5, ans3[c5]), (c7, ans3[c7]), (c9, ans3[c9]), (c1, ans3[c1])])

    r_      = sp.Symbol('r')
    pa      = sp.lambdify(r_, Pj[0], 'numpy')
    diffpa  = sp.diff(Pj[0], r_)
    Diffpa  = sp.lambdify(r_, diffpa, 'numpy')

    r_      = sp.Symbol('r')
    pc      = sp.lambdify(r_, Pj[1])
    diffpc  = sp.diff(Pj[1], r_)
    Diffpc  = sp.lambdify(r_, diffpc, 'numpy')

    r_      = sp.Symbol('r')
    pv      = sp.lambdify(r_, Pj[2])
    diffpv  = sp.diff(Pj[2], r_)
    Diffpv  = sp.lambdify(r_, diffpv, 'numpy')

    r_      = sp.Symbol('r')
    pe      = sp.lambdify(r_, Pj[3])
    diffpe  = sp.diff(Pj[3], r_)
    Diffpe  = sp.lambdify(r_, diffpe, 'numpy')

    r_      = sp.Symbol('r')
    pU      = sp.lambdify(r_, u[0], 'numpy')

    r       = np.linspace(0.03, 0.1, 10000)
    dr      = sp.N((0.1 - 0.03) / 9999, digs)
    
#     PA_pred = sp.Matrix(pa(r)).evalf(digs)
#     PC_pred = sp.Matrix(pc(r)).evalf(digs)
#     PV_pred = sp.Matrix(pv(r)).evalf(digs)
#     PE_pred = sp.Matrix(pe(r)).evalf(digs)
    PA_pred = pa(r)
    PC_pred = pc(r)
    PV_pred = pv(r)
    PE_pred = pe(r)
    
    Std_pa  = np.std(np.reshape(PA_ana, (10000)) - PA_pred)
    Std_pc  = np.std(np.reshape(PC_ana, (10000)) - PC_pred)
    Std_pv  = np.std(np.reshape(PV_ana, (10000)) - PV_pred)
    Std_pe  = np.std(np.reshape(PE_ana, (10000)) - PE_pred)
    Mean_pa = np.mean(np.abs(np.reshape(PA_ana, (10000)) - PA_pred))
    Mean_pc = np.mean(np.abs(np.reshape(PC_ana, (10000)) - PC_pred))
    Mean_pv = np.mean(np.abs(np.reshape(PV_ana, (10000)) - PV_pred))
    Mean_pe = np.mean(np.abs(np.reshape(PE_ana, (10000)) - PE_pred))

    STD     = [Std_pa, Std_pc, Std_pv, Std_pe]
    return STD

Main

In [ ]:
if __name__ == '__main__':
    digs        = 50
    mem1        = memory_profiler.memory_usage()
    r_old       = np.linspace(0.03, 0.1, 1000)
    r           = np.linspace(0.03, 0.1, 10000)
    dr          = np.float((0.1 - 0.03) / 9999)
    Dat         = np.empty([10000, 4])
    ppa         = np.loadtxt('/Users/jasonwang/Desktop/Lab/datasets/pa.dat')
    ppc         = np.loadtxt('/Users/jasonwang/Desktop/Lab/datasets/pc.dat')
    ppv         = np.loadtxt('/Users/jasonwang/Desktop/Lab/datasets/pv.dat')
    ppe         = np.loadtxt('/Users/jasonwang/Desktop/Lab/datasets/pe.dat')
#     ppa         = np.loadtxt("SSPArt 00000000000000000.dat") # Change to analytical data
#     ppc         = np.loadtxt("SSPCap 00000000000000000.dat")
#     ppv         = np.loadtxt("SSPVen 00000000000000000.dat")
#     ppe         = np.loadtxt("SSPCsf 00000000000000000.dat")
    Dat[:, 0:1] = np.reshape(ppa, (10000, 1))
    Dat[:, 1:2] = np.reshape(ppc, (10000, 1))
    Dat[:, 2:3] = np.reshape(ppv, (10000, 1))
    Dat[:, 3:4] = np.reshape(ppe, (10000, 1))
#     Dat[:, 0:1] = np.reshape(np.interp(r, r_old, np.reshape(ppa[:, 1:], (1000, ))), (10000, 1))
#     Dat[:, 1:2] = np.reshape(np.interp(r, r_old, np.reshape(ppc[:, 1:], (1000, ))), (10000, 1))
#     Dat[:, 2:3] = np.reshape(np.interp(r, r_old, np.reshape(ppv[:, 1:], (1000, ))), (10000, 1))
#     Dat[:, 3:4] = np.reshape(np.interp(r, r_old, np.reshape(ppe[:, 1:], (1000, ))), (10000, 1))
    P           = Dat.T
    epoch       = 0
    epochs      = 1000
    Num         = 10000
    epsilon     = 1.0e-6
    epsilon_std = 1.0e-6
    ka          = 3.7e-8
    kc          = 3.7e-8
    kv          = 3.7e-8
    ke          = 1.6e-11
    r_ac        = None
    r_ca        = None
    r_cv        = None
    r_vc        = None
    r_ce        = None
    r_ec        = None
    r_ve        = None
    r_ev        = None
    gamma       = [r_ac, r_ca, r_cv, r_vc, r_ce, r_ec, r_ve, r_ev]
    grad_ac     = None
    grad_ca     = None
    grad_cv     = None
    grad_vc     = None
    grad_ce     = None
    grad_ec     = None
    grad_ve     = None
    grad_ev     = None
    grad        = [grad_ac, grad_ca, grad_cv, grad_vc, grad_ce, grad_ec, grad_ve, grad_ev]
    n           = [      0,       0,       0,       0,       0,       0,       0,       0]
    e           = 0
    Gamma, Grad = train(gamma, grad, n, epoch)
    STD         = test(Gamma)
    
    i           = 0
    while i == 0:
        e               = e + 1
        epsilon         = 10**(- 0.1 * e - 6) #Change the iteration rate
        if np.greater(STD[0], epsilon_std) or np.greater(STD[1], epsilon_std) or np.greater(STD[2], epsilon_std) or np.greater(STD[3], epsilon_std):
            Gamma, Grad = train(Gamma, Grad, n, epoch)
            STD         = test(Gamma)
            
        else:
            i = 1

Iteration 0
[8.693833005420585e-11, 8.693833005420585e-11, 2.830647776820061e-11, 2.830647776820061e-11, 1.3750517764856063e-10, 1.3750517764856063e-10, 1.4955990945278612e-10, 1.4955990945278612e-10]
epsilon =  1e-06 1
1
0 0 0 0
It took 2.8888309001922607 Secs and 1.1015625 Mb to execute this method
Iteration 1
[7.930238489221161e-11, 7.930238489221161e-11, 7.123108787056261e-11, 7.123108787056261e-11, 1.3750517764856063e-10, 1.3750517764856063e-10, 1.4955990945278612e-10, 1.4955990945278612e-10]
epsilon =  7.943282347242822e-07 58
58
57 57 18 0
It took 21.182158708572388 Secs and 20.4375 Mb to execute this method
Iteration 2
[6.256548560088656e-11, 6.256548560088656e-11, 7.123108787056261e-11, 7.123108787056261e-11, 1.3750517764856063e-10, 1.3750517764856063e-10, 1.4955990945278612e-10, 1.4955990945278612e-10]
epsilon =  6.30957344480193e-07 24
24
23 23 0 0
It took 9.766466856002808 Secs and 36.4453125 Mb to execute this method
Iteration 3
[4.991734042423756e-11, 4.991734042423756e-1